In [1]:
import tensorflow as tf

In [2]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

In [3]:
# creating a tf dataset from list

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [4]:
# itration through tf dataset
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [5]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [6]:
for sales in tf_dataset.take(5):
    print(sales.numpy())

21
22
-108
31
-1


In [7]:
tf_dataset=tf_dataset.filter(lambda x: x>0)

for sales in tf_dataset:
    print(sales.numpy())

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
21
22
31
32
34
31


In [8]:
# Convert sales numbers from USA dollars ($) to Indian Rupees (INR) Assuming 1->80 conversation rate
tf_dataset = tf_dataset.map(lambda x: x*80)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1680
1760
2480
2560
2720
2480


In [9]:
tf_dataset = tf_dataset.shuffle(3)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1760
2480
2720
1680
2480
2560


In [10]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1760 2560]
[2480 1680]
[2480 2720]


In [11]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*80).shuffle(2).batch(2)

for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1760 2480]
[2560 2720]
[1680 2480]


<b style="color:lightgreen"><i>Applying the above operations on Image dataset</i></b>

In [12]:
images = tf.data.Dataset.list_files('./images/*/*', shuffle=False)

In [13]:
img_count = len(images)
img_count

130

In [14]:
type(images)

tensorflow.python.data.ops.from_tensor_slices_op.TensorSliceDataset

In [15]:
for img in images.take(5):
    print(img.numpy())

b'.\\images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg'
b'.\\images\\cat\\7 Foods Your Cat Can_t Eat.jpg'
b'.\\images\\cat\\A cat appears to have caught the....jpg'
b'.\\images\\cat\\Adopt-A-Cat Month\xc2\xae - American Humane....jpg'
b'.\\images\\cat\\All About Your Cat_s Tongue.jpg'


In [16]:
images = images.shuffle(130)

for img in images.take(5):
    print(img.numpy())

b'.\\images\\dog\\Great Dane - Wikipedia.jpg'
b'.\\images\\dog\\Welcoming Your Adopted Dog Into Your....jpg'
b'.\\images\\dog\\List of dog breeds - Wikipedia.jpg'
b'.\\images\\dog\\Canine Mind....jpg'
b'.\\images\\dog\\Mood-Boosting Benefits of Pets....jpg'


In [17]:
classes = ['Cat', 'Dog']

In [18]:
train_size = int(img_count*0.8)
train_ds = images.take(train_size)
test_ds = images.skip(train_size)

In [19]:
len(train_ds), len(test_ds)

(104, 26)

In [20]:
import os

def get_class_label(path):
    parts = tf.strings.split(path, os.path.sep)
    return parts[-2]

In [21]:
get_class_label(".\\images\\dog\\Hong Kong dog causes panic \xe2\x80\x93 but here_s... (1).jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'dog'>

In [22]:
def image_processing(path):
    class_label=get_class_label(path)
    
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    
    return img, class_label

In [23]:
img, label = image_processing('.\\images\\cat\\International Cat Care _ The ultimate....jpg')
print(img.numpy())
print(label.numpy())

[[[218.03906 218.03906 218.03906]
  [219.5625  219.5625  219.5625 ]
  [219.      219.      219.     ]
  ...
  [226.      231.      235.     ]
  [226.      231.      235.     ]
  [225.      230.      234.     ]]

 [[219.11719 219.11719 219.11719]
  [219.5625  219.5625  219.5625 ]
  [219.      219.      219.     ]
  ...
  [226.      231.      235.     ]
  [226.      231.      235.     ]
  [225.      230.      234.     ]]

 [[219.      219.      219.     ]
  [219.      219.      219.     ]
  [219.      219.      219.     ]
  ...
  [227.      232.      236.     ]
  [226.      231.      235.     ]
  [225.      230.      234.     ]]

 ...

 [[210.      205.      201.     ]
  [210.      205.      201.     ]
  [210.      205.      201.     ]
  ...
  [201.      194.      186.39062]
  [203.80469 195.19531 190.     ]
  [201.8125  196.8125  190.8125 ]]

 [[210.      205.      201.     ]
  [210.      205.      201.     ]
  [210.      205.      201.     ]
  ...
  [201.      194.      188.     ]
  [2

In [24]:
def scaling(image, label):
    return image/255, label

In [25]:
train_ds = train_ds.map(image_processing)
test_ds = test_ds.map(image_processing)

In [26]:
train_ds = train_ds.map(scaling)
test_ds = test_ds.map(scaling)

In [27]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[0.9059021  0.9137452  0.85884327]
  [0.9098039  0.91764706 0.8627451 ]
  [0.91764706 0.9254902  0.87058824]
  ...
  [0.85518616 0.8630293  0.76891166]
  [0.8523417  0.86018485 0.7660672 ]
  [0.8619026  0.86974573 0.7873928 ]]

 [[0.9089614  0.91680455 0.8619026 ]
  [0.90588236 0.9137255  0.85882354]
  [0.9137255  0.92156863 0.8666667 ]
  ...
  [0.85435563 0.86219877 0.7753831 ]
  [0.853606   0.8614491  0.7673315 ]
  [0.85490197 0.8627451  0.76862746]]

 [[0.90811884 0.915962   0.86106   ]
  [0.907113   0.91495615 0.8600542 ]
  [0.90588236 0.9137255  0.85882354]
  ...
  [0.8473499  0.855193   0.7689185 ]
  [0.85068417 0.8585273  0.76440966]
  [0.8509804  0.85882354 0.7647059 ]]

 ...

 [[0.8745864  0.88242954 0.8275276 ]
  [0.88235295 0.8901961  0.8352941 ]
  [0.88973653 0.89757967 0.8426777 ]
  ...
  [0.85085785 0.8547794  0.79929537]
  [0.8476     0.85152155 0.7859885 ]
  [0.8509804  0.85490197 0.7912378 ]]

 [[0.87758887 0.885432   0.83053005]
  [0.88235295 0.89019

In [28]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.99607843 0.99607843 0.99607843]
****Label:  b'dog'
****Image:  [0.9529412  0.9607843  0.95686275]
****Label:  b'dog'
****Image:  [0.10965074 0.1645527  0.03097427]
****Label:  b'dog'
****Image:  [0.8432598  0.8079657  0.74129903]
****Label:  b'dog'
****Image:  [0.7567622  0.831272   0.61558574]
****Label:  b'dog'
